# Assignment_8.21442

### Read the following data set:

https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [2]:
import pandas as pd
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, Float, Date
from sqlalchemy.ext.declarative import declarative_base

#### 1. Create an sqlalchemy engine using a sample from the data set

In [3]:
adult_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", 
                       names=['age','workclass','fnlwgt','education','education_num',
                              'marital_status','occupation','relationship','race','sex',
                              'capital_gain','capital_loss','hours_per_week','native_country','income_range'])

In [52]:
# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file
engine = create_engine("sqlite:///:memory:")
adult_df.to_sql("adult", engine)

#Bind the engine to the metadata of the Base class so that the declaratives can be accessed 
conn = engine.connect()
metadata = sqlalchemy.MetaData(bind=engine)
adult = sqlalchemy.Table('adult', metadata, autoload=True, autoload_with=engine)
adult.columns.keys()

['index',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'income_range']

#### 2. Write two basic update queries

In [60]:
q1 = sqlalchemy.select([adult])
conn.execute(q1).fetchall()

[(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', 'M', 2174, 0, 40, 'USA', ' <=50K'),
 (1, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', 'M', 0, 0, 13, 'USA', ' <=50K'),
 (2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', 'M', 0, 0, 40, 'USA', ' <=50K'),
 (3, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', 'M', 0, 0, 40, 'USA', ' <=50K'),
 (4, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K'),
 (5, 37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, 'USA', ' <=50K'),
 (6, 49, ' Private', 160187, ' 9th', 5, ' Married-spouse-absent', ' Other-service', ' Not-in-family', ' Black', ' Female', 0

In [63]:
engine.execute("SELECT * FROM adult limit 5").fetchall()

[(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', 'M', 2174, 0, 40, 'USA', ' <=50K'),
 (1, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', 'M', 0, 0, 13, 'USA', ' <=50K'),
 (2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', 'M', 0, 0, 40, 'USA', ' <=50K'),
 (3, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', 'M', 0, 0, 40, 'USA', ' <=50K'),
 (4, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K')]

In [11]:
q = sqlalchemy.select([adult]).where(sqlalchemy.and_(adult.columns['age'].is_(30), adult.columns.fnlwgt.is_(141297)))
conn.execute(q).fetchall()

[(11, 30, ' State-gov', 141297, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' India', ' >50K')]

In [12]:
engine.execute("UPDATE adult set education = ' Masters' where age=30 and fnlwgt=141297")
engine.execute("SELECT * from adult where age=30 and fnlwgt=141297").fetchall()

[(11, 30, ' State-gov', 141297, ' Masters', 13, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' India', ' >50K')]

In [13]:
q = sqlalchemy.update(adult).values(education_num = 14)
q = q.where(sqlalchemy.and_(adult.columns['age'] == 30, adult.columns.fnlwgt.is_(141297)))
conn.execute(q)

q1 = sqlalchemy.select([adult]).where(sqlalchemy.and_(adult.columns['age'] == 30, adult.columns.fnlwgt == 141297))
conn.execute(q1).fetchall()

[(11, 30, ' State-gov', 141297, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' Asian-Pac-Islander', ' Male', 0, 0, 40, ' India', ' >50K')]

#### 3. Write two delete queries

In [15]:
engine.execute("SELECT * from adult where age>=90 and marital_status = ' Widowed' and relationship =' Not-in-family'").fetchall()


[(8963, 90, ' ?', 77053, ' HS-grad', 9, ' Widowed', ' ?', ' Not-in-family', ' White', ' Female', 0, 4356, 40, ' United-States', ' <=50K'),
 (11731, 90, ' ?', 39824, ' HS-grad', 9, ' Widowed', ' ?', ' Not-in-family', ' White', ' Male', 401, 0, 4, ' United-States', ' <=50K'),
 (24238, 90, ' ?', 166343, ' 1st-4th', 2, ' Widowed', ' ?', ' Not-in-family', ' Black', ' Female', 0, 0, 40, ' United-States', ' <=50K')]

In [17]:
engine.execute("""DELETE from adult where age>=90 
                    and marital_status = ' Widowed' 
                    and relationship =' Not-in-family' 
                    and fnlwgt = 77053""")

engine.execute("SELECT * from adult where age>=90 and marital_status = ' Widowed' and relationship =' Not-in-family'").fetchall()

[(11731, 90, ' ?', 39824, ' HS-grad', 9, ' Widowed', ' ?', ' Not-in-family', ' White', ' Male', 401, 0, 4, ' United-States', ' <=50K'),
 (24238, 90, ' ?', 166343, ' 1st-4th', 2, ' Widowed', ' ?', ' Not-in-family', ' Black', ' Female', 0, 0, 40, ' United-States', ' <=50K')]

In [22]:
q = sqlalchemy.delete(adult).where(sqlalchemy.and_(adult.columns.age == 90, 
                                                   adult.columns.marital_status == ' Widowed', 
                                                   adult.columns.relationship == ' Not-in-family',
                                                   adult.columns.fnlwgt == 166343
                                                  ))
conn.execute(q)

q1 = sqlalchemy.select([adult]).where(sqlalchemy.and_(adult.columns.age == 90, 
                                                   adult.columns.marital_status == ' Widowed', 
                                                   adult.columns.relationship == ' Not-in-family',
                                                  ))
conn.execute(q1).fetchall()

[(11731, 90, ' ?', 39824, ' HS-grad', 9, ' Widowed', ' ?', ' Not-in-family', ' White', ' Male', 401, 0, 4, ' United-States', ' <=50K')]

#### 4. Write two filter queries

In [33]:
#filter where age=29 and 30
conn = engine.connect()
result = conn.execute("SELECT distinct(age) FROM adult where age>0 and age<100 order by age asc").fetchall()
print(result)

[(17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (84,), (85,), (86,), (87,), (88,), (90,)]


In [45]:
# where profit<5000
conn = engine.connect()
result = conn.execute("SELECT count(capital_gain-capital_loss) FROM adult where capital_gain-capital_loss>5000").fetchall()
print(result)

[(1648,)]


#### 5. Write two function queries

In [46]:
from sqlalchemy import func,select,Column,Table,Integer

In [47]:
session = sessionmaker(bind=engine)
s = session()
s.query(adult).filter(func.length(adult.columns.education_num).is_(0)).all()

[]

In [48]:
engine.execute("select * from adult where LENGTH(education_num)==0 ").fetchall()

[]

In [49]:
session = sessionmaker(bind=engine)
s = session()
s.query(func.max(adult.columns.age)).all()

[(90)]

In [50]:
engine.execute("select max(age) from adult").fetchall()

[(90,)]